In [ ]:
!pip install transformers
!pip install pycaret

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import transformers
from transformers import AutoModel,BertTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
#specify gpu
device=torch.device('cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/projects


/content/drive/MyDrive/Colab Notebooks/projects


In [ ]:
true_data=pd.read_csv('a1_True.csv')
fake_data=pd.read_csv('a2_Fake.csv')

true_data['Target']=['True']*len(true_data)
fake_data['Target']=['Fake']*len(fake_data)
#we concate the 2 datasets and shuffle them into single datset data
data= pd.concat([true_data,fake_data]).sample(frac=1).reset_index(drop=True)
print(data.shape)
data.head()

(44898, 5)


,title,text,subject,date,Target
0,WOMAN DEFENDS Female Genital Mutilation With T...,Anthropologist Fuambai Ahmadu appeared on Tuck...,left-news,"May 4, 2017",Fake
1,WHOA! ‘SESAME STREET’ Using Bert And Ernie For...,Wrong target audience? What the heck is going ...,left-news,"May 3, 2016",Fake
2,IT JUST GOT REAL! GOP Rep. Jim Jordan Tells Ju...,One of the big players in trying to get to the...,left-news,"Dec 17, 2017",Fake
3,TRUMP’S DOJ MAKES Announcement on Anti-Gun Oba...,The program is dead! Operation Choke Point is ...,politics,"Aug 18, 2017",Fake
4,Thai university removes student leader for def...,BANGKOK (Reuters) - Thailand s prestigious Chu...,worldnews,"September 1, 2017",True


In [ ]:
# prompt: convert the true or false into 0,1

data['label']=pd.get_dummies(data['Target'])['Fake']
data.head()

,title,text,subject,date,Target,label
0,WOMAN DEFENDS Female Genital Mutilation With T...,Anthropologist Fuambai Ahmadu appeared on Tuck...,left-news,"May 4, 2017",Fake,True
1,WHOA! ‘SESAME STREET’ Using Bert And Ernie For...,Wrong target audience? What the heck is going ...,left-news,"May 3, 2016",Fake,True
2,IT JUST GOT REAL! GOP Rep. Jim Jordan Tells Ju...,One of the big players in trying to get to the...,left-news,"Dec 17, 2017",Fake,True
3,TRUMP’S DOJ MAKES Announcement on Anti-Gun Oba...,The program is dead! Operation Choke Point is ...,politics,"Aug 18, 2017",Fake,True
4,Thai university removes student leader for def...,BANGKOK (Reuters) - Thailand s prestigious Chu...,worldnews,"September 1, 2017",True,False


In [ ]:
data["label"]=data["label"].astype(int)

In [ ]:
data.head()

,title,text,subject,date,Target,label
0,WOMAN DEFENDS Female Genital Mutilation With T...,Anthropologist Fuambai Ahmadu appeared on Tuck...,left-news,"May 4, 2017",Fake,1
1,WHOA! ‘SESAME STREET’ Using Bert And Ernie For...,Wrong target audience? What the heck is going ...,left-news,"May 3, 2016",Fake,1
2,IT JUST GOT REAL! GOP Rep. Jim Jordan Tells Ju...,One of the big players in trying to get to the...,left-news,"Dec 17, 2017",Fake,1
3,TRUMP’S DOJ MAKES Announcement on Anti-Gun Oba...,The program is dead! Operation Choke Point is ...,politics,"Aug 18, 2017",Fake,1
4,Thai university removes student leader for def...,BANGKOK (Reuters) - Thailand s prestigious Chu...,worldnews,"September 1, 2017",True,0


In [ ]:
data["Target"].value_counts()

,count
Target,
Fake,23481
True,21417


In [ ]:
# Train-Validation-Test set split into 70:15:15 ratio
#Train-Temp split
#we are splitting training and temp in 70:30
train_text,temp_text,train_labels,temp_labels =train_test_split(
    data['title'],data['label'],random_state=2018,test_size=0.3,stratify=data['label'])
#we are spliting temp in validation and test
#validation test split
val_text,test_text,val_labels,test_labels =train_test_split(
    temp_text,temp_labels,random_state=2018,test_size=0.5,stratify=temp_labels)

In [ ]:
#Loading pretrained bert model

In [ ]:
bert=AutoModel.from_pretrained('bert-base-cased')
tokenizer=BertTokenizerFast.from_pretrained('bert-base-cased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
#checking the funcitonality of bert tokenizer
sample_data=["build fake news model.","using bert."]

tokenized_sample_data=tokenizer.batch_encode_plus(sample_data,padding=True)
print(tokenized_sample_data['input_ids'])
print(tokenized_sample_data['attention_mask'])

#in attention mask 1->pay attention to this token 0->ignore this token
#101 -[cls]  102 -[sep]

[[101, 3076, 8406, 2371, 2235, 119, 102], [101, 1606, 1129, 3740, 119, 102, 0]]
[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0]]


In [ ]:
#we are training the model using titles due to computational deficiency
# so we are tokenizing only titles
MAX_LENGTH =15 #majority titles have word length of 15

tokens_train=tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length=MAX_LENGTH,
    pad_to_max_length=True,
    truncation=True,
)
tokens_val=tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length=MAX_LENGTH,
    pad_to_max_length=True,
    truncation=True,
)
tokens_test=tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length=MAX_LENGTH,
    pad_to_max_length=True,
    truncation=True,
)

In [ ]:
#converting lists into tensors

train_seq=torch.tensor(tokens_train['input_ids'])
train_mask=torch.tensor(tokens_train['attention_mask'])
train_y=torch.tensor(train_labels.tolist())

val_seq=torch.tensor(tokens_val['input_ids'])
val_mask=torch.tensor(tokens_val['attention_mask'])
val_y=torch.tensor(val_labels.tolist())

test_seq=torch.tensor(tokens_test['input_ids'])
test_mask=torch.tensor(tokens_test['attention_mask'])
test_y=torch.tensor(test_labels.tolist())

In [ ]:
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
batch_size=32
train_data=TensorDataset(train_seq,train_mask,train_y)
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

val_data=TensorDataset(val_seq,val_mask,val_y)
val_sampler=RandomSampler(val_data)
val_dataloader=DataLoader(val_data,sampler=val_sampler,batch_size=batch_size)

In [ ]:
for param in bert.parameters(): #freezing the parameters
  param.requires_grad=False

In [ ]:
class BERT_Arch(nn.Module):
  def __init__(self,bert):
    super(BERT_Arch,self).__init__()
    self.bert=bert
    self.dropout=nn.Dropout(0.1)
    self.relu=nn.ReLU()
    self.fc1=nn.Linear(768,512)
    self.fc2=nn.Linear(512,2)
    self.softmax=nn.LogSoftmax(dim=1)

  def forward(self,sent_id,mask):
    cls_hs=self.bert(sent_id,attention_mask=mask)['pooler_output']
    x=self.fc1(cls_hs)
    x=self.relu(x)
    x=self.dropout(x)
    x=self.fc2(x)
    x=self.softmax(x)
    return x

model=BERT_Arch(bert)
#defining the hyperparameters(optimizer,weights,epochs)
from torch.optim import AdamW


optimizer=AdamW(model.parameters(),lr=1e-5)  #defining the optimizer
#defining the loss function
cross_entropy=nn.NLLLoss()
epochs=2

In [ ]:
# Defining training and evaluation functions
def train():
  model.train()
  total_loss, total_accuracy = 0, 0

  for step,batch in enumerate(train_dataloader):                # iterate over batches
    if step % 50 == 0 and not step == 0:                        # progress update after every 50 batches.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
    batch = [r for r in batch]                                  # push the batch to gpu
    sent_id, mask, labels = batch
    model.zero_grad()                                           # clear previously calculated gradients
    preds = model(sent_id, mask)                                # get model predictions for current batch
    loss = cross_entropy(preds, labels)                         # compute loss between actual & predicted values
    total_loss = total_loss + loss.item()                       # add on to the total loss
    loss.backward()                                             # backward pass to calculate the gradients
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)     # clip gradients to 1.0. It helps in preventing exploding gradient problem
    optimizer.step()                                            # update parameters
    preds=preds.detach().cpu().numpy()                          # model predictions are stored on GPU. So, push it to CPU

  avg_loss = total_loss / len(train_dataloader)                 # compute training loss of the epoch
                                                                # reshape predictions in form of (# samples, # classes)
  return avg_loss                                 # returns the loss and predictions

def evaluate():
  print("\nEvaluating...")
  model.eval()                                    # Deactivate dropout layers
  total_loss, total_accuracy = 0, 0
  for step,batch in enumerate(val_dataloader):    # Iterate over batches
    if step % 50 == 0 and not step == 0:          # Progress update every 50 batches.
                                                  
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
                                                  # Report progress
    batch = [t for t in batch]                    # Push the batch to GPU
    sent_id, mask, labels = batch
    with torch.no_grad():                         # Deactivate autograd
      preds = model(sent_id, mask)                # Model predictions
      loss = cross_entropy(preds,labels)          # Compute the validation loss between actual and predicted values
      total_loss = total_loss + loss.item()
      preds = preds.detach().cpu().numpy()
  avg_loss = total_loss / len(val_dataloader)         # compute the validation loss of the epoch
  return avg_loss

In [ ]:
#fine tuning bert model
best_valid_loss=float('inf')
train_losses=[]
valid_losses=[]
for epoch in range(epochs):
  print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
  train_loss=train()
  valid_loss=evaluate()

  if valid_loss <best_valid_loss:
    best_valid_loss=valid_loss
    torch.save(model.state_dict(),'saved_weights.pt')
  train_losses.append(train_loss)
  valid_losses.append(valid_loss)
  print(f'\nTraining Loss: {train_loss:.3f}')
  print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 2
  Batch    50  of    983.
  Batch   100  of    983.
  Batch   150  of    983.
  Batch   200  of    983.
  Batch   250  of    983.
  Batch   300  of    983.
  Batch   350  of    983.
  Batch   400  of    983.
  Batch   450  of    983.
  Batch   500  of    983.
  Batch   550  of    983.
  Batch   600  of    983.
  Batch   650  of    983.
  Batch   700  of    983.
  Batch   750  of    983.
  Batch   800  of    983.
  Batch   850  of    983.
  Batch   900  of    983.
  Batch   950  of    983.

Evaluating...
  Batch    50  of    211.
  Batch   100  of    211.
  Batch   150  of    211.
  Batch   200  of    211.

Training Loss: 0.565
Validation Loss: 0.440

 Epoch 2 / 2
  Batch    50  of    983.
  Batch   100  of    983.
  Batch   150  of    983.
  Batch   200  of    983.
  Batch   250  of    983.
  Batch   300  of    983.
  Batch   350  of    983.
  Batch   400  of    983.
  Batch   450  of    983.
  Batch   500  of    983.
  Batch   550  of    983.
  Batch   600  of    983.
  

In [ ]:
path='saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
with torch.no_grad():
  preds=model(test_seq,test_mask)
  preds=preds.detach().cpu().numpy()
preds=np.argmax(preds,axis=1)
print(classification_report(test_y,preds))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      3212
           1       0.95      0.95      0.95      3523

    accuracy                           0.95      6735
   macro avg       0.95      0.95      0.95      6735
weighted avg       0.95      0.95      0.95      6735



In [ ]:
unseen_news_text = ["Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing",     # Fake
                    "WATCH: George W. Bush Calls Out Trump For Supporting White Supremacy",               # Fake
                    "U.S. lawmakers question businessman at 2016 Trump Tower meeting: sources",           # True
                    "Trump administration issues new rules on U.S. visa waivers"                          # True
                    ]
MAX_LENGTH=15
tokens_unseen=tokenizer.batch_encode_plus(
    unseen_news_text,
    max_length=MAX_LENGTH,
    pad_to_max_length=True,
    truncation=True,
)

unseen_seq=torch.tensor(tokens_unseen['input_ids'])
unseen_mask=torch.tensor(tokens_unseen['attention_mask'])
with torch.no_grad():
  preds=model(unseen_seq,unseen_mask)
  preds=preds.detach().cpu().numpy()
preds=np.argmax(preds,axis=1)
print(preds)

[1 1 0 0]
